In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/My Drive/589MiniProject/")
%cd /content/drive/My Drive/589MiniProject/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1yyh1nEBAISidzfrj3Aql57NwD7xFpa20/589MiniProject


In [2]:
import pandas as pd
import numpy as np
from math import ceil

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn import preprocessing

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 75

print('using device:', device)

using device: cpu


In [5]:
!pip install img2vec_pytorch

  Created wheel for img2vec-pytorch: filename=img2vec_pytorch-0.2.5-py3-none-any.whl size=4345 sha256=03cf04a0ab212d3043499c0adecbf94e07016213de9c406607ff3f41e94916db
  Stored in directory: /root/.cache/pip/wheels/07/4d/bf/9de03c3e76c20899b69498f90624ec499c7c67dd13f183f4e9
Successfully built img2vec-pytorch


In [6]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# # Initialize Img2Vec with GPU
# img2vec = Img2Vec(torch.cuda.is_available())

# # Read in an image (rgb format)
# img = Image.open('test.jpg')
# # Get a vector from img2vec, returned as a torch FloatTensor
# vec = img2vec.get_vec(img, tensor=True)
# # Or submit a list
# vectors = img2vec.get_vec(list_of_PIL_images)
def imgVec(folder='Train'):
  dataPath = "Data/" + folder + "/"
  metadataPath = "Data/MetaData/" + folder.lower() + ".csv"
  df = pd.read_csv(metadataPath)
  df['img_path'] = './Data/' + folder + "/" + df['Id'] + '.jpg'
  img2vec = Img2Vec(cuda=(torch.cuda.is_available()))  
  img_vecs = []
  for idx, row in df.iterrows():
    # print(path + '/Data/' + 'Test/' + row['Id'] + '.jpg')
    img = Image.open(row["img_path"])
    vec = img2vec.get_vec(img)
    img_vecs.append(vec)

  df['feature_vector'] = img_vecs
  return df


In [7]:
# test_df = imgVec('Test')
# test_df.reset_index(inplace=True)
# test_df.head()

In [8]:
# print(len(test_df.iloc[0]["feature_vector"]))

# col_names = ["feat"+str(i) for i in range(len(test_df.iloc[0]["feature_vector"]))]

# test_df[col_names] = pd.DataFrame(test_df.feature_vector.tolist(), index= test_df.index)

# final_testdf = test_df.copy()
# final_testdf.drop(columns = ["Id","img_path","feature_vector"], inplace=True)

# final_testdf.head()

In [9]:
train_df = imgVec('Train')
train_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(train_df.iloc[0]["feature_vector"]))]
train_df[col_names] = pd.DataFrame(train_df.feature_vector.tolist(), index= train_df.index)
train_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)


valid_df = imgVec('Valid')
valid_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(valid_df.iloc[0]["feature_vector"]))]
valid_df[col_names] = pd.DataFrame(valid_df.feature_vector.tolist(), index= valid_df.index)
valid_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)

test_df = imgVec('Test')
test_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(test_df.iloc[0]["feature_vector"]))]
test_df[col_names] = pd.DataFrame(test_df.feature_vector.tolist(), index= test_df.index)
test_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)

print("Train ...")
print(train_df.head())

print("Valid ...")
print(valid_df.head())

print("Test ...")
print(test_df.head())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Train ...
   index  Subject Focus  Eyes  Face  ...   feat508   feat509   feat510   feat511
0      0              0     1     1  ...  1.552251  0.729470  1.408372  1.468240
1      1              0     1     1  ...  0.841265  1.340379  2.015083  0.379474
2      2              0     1     1  ...  0.902725  0.322946  1.937619  0.461102
3      3              0     1     1  ...  1.789118  1.036564  0.068066  0.716284
4      4              0     1     1  ...  3.102909  1.097454  0.581716  0.273106

[5 rows x 526 columns]
Valid ...
   index  Subject Focus  Eyes  Face  ...   feat508   feat509   feat510   feat511
0      0              0     0     1  ...  1.847430  0.734269  0.584828  1.115096
1      1              0     1     1  ...  1.499832  2.500482  0.277388  0.397272
2      2              0     1     1  ...  0.401953  0.236270  0.914187  0.319195
3      3              0     1     1  ...  2.246782  1.693573  0.013458  1.673211
4      4              0     1     1  ...  2.828182  1.301023  0.4

In [10]:
def hyper_parameter_tuning(model_class, param_grid, train_X, train_y, valid_X, valid_y):
    """
    Tune the hyper-parameter using training and validation data

    Params
    ------
    model_class:    the model class
    param_grid:     the hyper-parameter grid, dict
    train:          the training data (train_X, train_y)
    valid:          the validatation data (valid_X, valid_y)

    Return
    ------
    model:          model fit with best params
    best_param:     the best params
    """

    # Set up the parameter grid
    param_grid = list(ParameterGrid(param_grid))
    best_model = None
    best_mse = float('inf')
    best_params = None

    # train the model with each parameter setting in the grid
    for params in param_grid:
        model = model_class(n_estimators=params['n_estimators'],learning_rate=params['learning_rate'])
        model.fit(train_X,train_y)
        y_pred = model.predict(valid_X)
        mse = mean_squared_error(valid_y,y_pred)
        print(params,mse)
        if mse < best_mse:
            best_model = model
            best_mse = mse
            best_params = params

    # choose the model with lowest MAE on validation set
    # then fit the model with the training and validation set (refit)

    full_training_set_X = np.concatenate([train_X, valid_X], axis=0)
    full_training_set_y = np.concatenate([train_y, valid_y], axis=0)
    best_model.fit(full_training_set_X,full_training_set_y)

    # return the fitted model and the best parameter setting
    return best_model, best_params

In [11]:
train_y, valid_y, test_y = train_df['Pawpularity'], valid_df['Pawpularity'], test_df['Pawpularity']
train_X = train_df.drop(columns=['Pawpularity'])
train_X = preprocessing.StandardScaler().fit_transform(train_X)
valid_X = valid_df.drop(columns=['Pawpularity'])
valid_X = preprocessing.StandardScaler().fit_transform(valid_X)
test_X = test_df.drop(columns=['Pawpularity'])
test_X = preprocessing.StandardScaler().fit_transform(test_X)

In [12]:
features = train_X.shape[1]
gb_model = GradientBoostingRegressor(random_state=0)
gb_model.fit(train_X, train_y)
validpred_y = gb_model.predict(valid_X)
testpred_y = gb_model.predict(test_X)
validmse = mean_squared_error(valid_y, validpred_y)
testmse = mean_squared_error(test_y, testpred_y)

print("Validation MSE ... ", validmse)
print("TEST MSE ... ", testmse)

Validation MSE ...  373.81287052741783
TEST MSE ...  343.3951248356901


In [13]:
gb_grid = dict(n_estimators=[30, 50, 70, 100],
                      learning_rate=[0.05,0.1,0.2,0.3])
gb_model, gb_param = hyper_parameter_tuning(GradientBoostingRegressor, gb_grid, train_X, train_y, valid_X, valid_y)
print('best parameter setting for Gradient Boosting:',gb_param)
testpred_y = gb_model.predict(test_X)
testmse = mean_squared_error(test_y, testpred_y)
print("TEST MSE ... ", testmse)
print("TEST RMSE:", np.sqrt(testmse))

{'learning_rate': 0.05, 'n_estimators': 30} 385.2280344984839
{'learning_rate': 0.05, 'n_estimators': 50} 377.3294654951674
{'learning_rate': 0.05, 'n_estimators': 70} 374.59131967426543
{'learning_rate': 0.05, 'n_estimators': 100} 371.76057795145806
{'learning_rate': 0.1, 'n_estimators': 30} 377.9469562180452
{'learning_rate': 0.1, 'n_estimators': 50} 374.0293836106946
{'learning_rate': 0.1, 'n_estimators': 70} 374.3949816745915
{'learning_rate': 0.1, 'n_estimators': 100} 374.21428885197815
{'learning_rate': 0.2, 'n_estimators': 30} 378.9534088600454
{'learning_rate': 0.2, 'n_estimators': 50} 379.9632284243963
{'learning_rate': 0.2, 'n_estimators': 70} 382.50956662691414
{'learning_rate': 0.2, 'n_estimators': 100} 385.7229561755681
{'learning_rate': 0.3, 'n_estimators': 30} 384.8286204519821
{'learning_rate': 0.3, 'n_estimators': 50} 390.8498140768882
{'learning_rate': 0.3, 'n_estimators': 70} 395.89603102939145
{'learning_rate': 0.3, 'n_estimators': 100} 413.259793405116
best paramet